# Minecraft Transformation Graph - Cosmograph Visualization

This notebook provides an interactive, GPU-accelerated visualization of Minecraft item transformations using [Cosmograph](https://cosmograph.app/), a high-performance graph visualization widget.

## Features

- **Interactive exploration**: Zoom, pan, and hover over nodes
- **Force-directed layout**: Physics-based simulation creates natural clustering
- **GPU-accelerated**: Smooth 60 FPS rendering even with 1000+ nodes
- **Color-coded edges**: Transformation types use different colors (crafting, smelting, etc.)
- **Smart node sizing**: More connected items appear larger
- **Directional arrows**: Shows the flow of transformations

## Prerequisites

Make sure you've installed the required dependencies:
```bash
uv add cosmograph jupyter
```

## Table of Contents

1. [Setup & Data Loading](#setup)
2. [Data Preparation](#data-preparation)
3. [Basic Visualization](#basic-visualization)
4. [Customization Options](#customization)
5. [Filtering Examples](#filtering)
6. [Export & Sharing](#export)

## Setup & Data Loading <a id="setup"></a>

First, let's import the required libraries and load our transformation data.

In [1]:
# Standard library imports
import sys
from pathlib import Path

# Third-party imports
import pandas as pd
from cosmograph import Cosmograph

# Add src directory to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import our data preparation utilities
from src.utils.cosmograph_data_prep import prepare_cosmograph_data

print("✓ All imports successful!")
print(f"✓ Project root: {project_root}")

✓ All imports successful!
✓ Project root: /Users/apprentyr/projects/minegraph


## Data Preparation <a id="data-preparation"></a>

Now we'll prepare the transformation data for visualization. This involves:

1. **Loading transformations** from CSV (with JSON array parsing)
2. **Creating nodes** for all unique items
3. **Generating intermediate nodes** for multi-input transformations
4. **Building edges** with proper directionality
5. **Computing node sizes** based on connectivity (degree)
6. **Applying colors** from the configuration file

In [2]:
# Define file paths
csv_path = str(project_root / "output" / "transformations.csv")
config_path = str(project_root / "config" / "graph_colors.txt")

# Prepare data for Cosmograph
print("Preparing data for visualization...\n")
points, links, color_config = prepare_cosmograph_data(csv_path, config_path)

print("\n" + "="*50)
print("Data Preparation Complete!")
print("="*50)

Preparing data for visualization...

INFO - Starting Cosmograph data preparation...
INFO - Loaded 12 color mappings from /Users/apprentyr/projects/minegraph/config/graph_colors.txt
INFO - Loaded 2320 transformations from /Users/apprentyr/projects/minegraph/output/transformations.csv
INFO - Built points DataFrame: 1324 items + 1140 intermediate nodes
INFO - Built links DataFrame: 5283 edges
INFO - Calculated node sizes based on connectivity
INFO - Data preparation complete!
INFO -   Total nodes: 2464 (1324 items + 1140 intermediate)
INFO -   Total edges: 5283

Data Preparation Complete!


### Data Summary

Let's examine what we've loaded:

In [3]:
# Display statistics
item_count = sum(points['node_type'] == 'item')
intermediate_count = sum(points['node_type'] == 'intermediate')
total_nodes = len(points)
total_edges = len(links)

print(f"📊 Graph Statistics:")
print(f"   Total Nodes: {total_nodes:,}")
print(f"   - Item Nodes: {item_count:,}")
print(f"   - Intermediate Nodes: {intermediate_count:,}")
print(f"   Total Edges: {total_edges:,}")
print(f"\n🎨 Transformation Types: {len(color_config)}")
for trans_type, color in sorted(color_config.items()):
    print(f"   - {trans_type}: {color}")

📊 Graph Statistics:
   Total Nodes: 2,464
   - Item Nodes: 1,324
   - Intermediate Nodes: 1,140
   Total Edges: 5,283

🎨 Transformation Types: 12
   - bartering: #D4AF37
   - blast_furnace: #E67E22
   - brewing: #9B59B6
   - composting: #27AE60
   - crafting: #4A90E2
   - grindstone: #34495E
   - mob_drop: #E74C3C
   - smelting: #E67E22
   - smithing: #95A5A6
   - smoker: #E67E22
   - stonecutter: #7F8C8D
   - trading: #F39C12


### Sample Data

Here's a peek at the structure of our nodes and edges:

In [4]:
print("Sample Item Nodes:")
display(points[points['node_type'] == 'item'].head(10))

print("\nSample Edges:")
display(links.head(10))

Sample Item Nodes:


,id,label,node_type,size,color
0,Acacia Boat,Acacia Boat,item,14.771213,#FFFFFF
1,Acacia Boat with Chest,Acacia Boat with Chest,item,13.010300,#FFFFFF
2,Acacia Button,Acacia Button,item,13.010300,#FFFFFF
3,Acacia Door,Acacia Door,item,13.010300,#FFFFFF
4,Acacia Fence,Acacia Fence,item,13.010300,#FFFFFF
5,Acacia Fence Gate,Acacia Fence Gate,item,13.010300,#FFFFFF
6,Acacia Hanging Sign,Acacia Hanging Sign,item,13.010300,#FFFFFF
7,Acacia Log,Acacia Log,item,16.020600,#FFFFFF
8,Acacia Planks,Acacia Planks,item,26.020600,#FFFFFF
9,Acacia Pressure Plate,Acacia Pressure Plate,item,13.010300,#FFFFFF



Sample Edges:


,source,target,transformation_type,color,arrows
0,Gold Ingot,Enchanted Book,bartering,#D4AF37,True
1,Gold Ingot,Iron Boots,bartering,#D4AF37,True
2,Gold Ingot,Splash Potion of Fire Resistance,bartering,#D4AF37,True
3,Gold Ingot,Potion of Fire Resistance,bartering,#D4AF37,True
4,Gold Ingot,Water Bottle,bartering,#D4AF37,True
5,Gold Ingot,Dried Ghast,bartering,#D4AF37,True
6,Gold Ingot,Iron Nugget,bartering,#D4AF37,True
7,Gold Ingot,Ender Pearl,bartering,#D4AF37,True
8,Gold Ingot,String,bartering,#D4AF37,True
9,Gold Ingot,Nether Quartz,bartering,#D4AF37,True


## Basic Visualization <a id="basic-visualization"></a>

Now for the main event! Let's create our interactive graph visualization.

### How to interact with the graph:
- **Pan**: Click and drag the background
- **Zoom**: Scroll with your mouse/trackpad
- **Hover**: Move your cursor over nodes to see labels
- **Select**: Click nodes to highlight them

The graph uses a force-directed layout, which means nodes will arrange themselves based on their connections. Items that are closely related will naturally cluster together.

In [5]:
# Create the Cosmograph visualization
graph = Cosmograph(
    # Data
    points=points,
    links=links,
    
    # Point (node) configuration
    point_id_by='id',
    point_label_by='label',
    point_size_by='size',
    point_color_by='color',
    
    # Link (edge) configuration
    link_source_by='source',
    link_target_by='target',
    link_color_by='color',
    link_arrows=True,  # Show directional arrows
    link_width=1.5,
    
    # Label settings
    show_hovered_point_label=True,  # Show label when hovering
    show_top_labels=True,  # Always show labels for important nodes
    show_top_labels_limit=50,  # Number of top labels to show
    
    # Visual appearance
    background_color='#1a1a1a',  # Dark background
    point_size_scale=2.0,  # Scale up node sizes for visibility
    scale_points_on_zoom=True,  # Keep nodes visible when zooming
    render_links=True,
    
    # Force simulation parameters
    simulation_repulsion=0.5,  # How much nodes push apart
    simulation_link_spring=0.8,  # How strongly edges pull nodes together
    simulation_link_distance=20,  # Ideal edge length
    simulation_friction=0.8,  # Slow down movement over time
    simulation_decay=1500,  # How long simulation runs (ms)
)

print("🎨 Rendering graph visualization...")
print("   (This may take a few moments for large graphs)\n")

# Display the graph
graph

🎨 Rendering graph visualization...
   (This may take a few moments for large graphs)



## Customization Options <a id="customization"></a>

You can customize the visualization by creating a new graph with different parameters. Here are some examples:

### Example 1: Lighter Background with Different Colors

In [11]:
# Create a variant with light background
graph_light = Cosmograph(
    points=points,
    links=links,
    point_id_by='id',
    point_label_by='label',
    point_size_by='size',
    point_color_by='color',
    link_source_by='source',
    link_target_by='target',
    link_color_by='color',
    link_arrows=True,
    background_color='#f5f5f5',  # Light gray background
    show_hovered_point_label=True,
    show_top_labels=True,
)

# Uncomment to display:
graph_light

### Example 2: More Spread Out Layout

In [6]:
# Create a more spread-out version
graph_spread = Cosmograph(
    points=points,
    links=links,
    point_id_by='id',
    point_label_by='label',
    point_size_by='size',
    point_color_by='color',
    link_source_by='source',
    link_target_by='target',
    link_color_by='color',
    link_arrows=True,
    background_color='#1a1a1a',
    simulation_repulsion=1.0,  # Increased repulsion
    simulation_link_distance=40,  # Longer edges
    show_hovered_point_label=True,
)

# Uncomment to display:
graph_spread

## Filtering Examples <a id="filtering"></a>

You can create filtered views of the graph by subsetting the data. This is useful for exploring specific transformation types or item chains.

### Filter 1: Show Only Crafting Transformations

In [9]:
# Filter links to only crafting
crafting_links = links[links['transformation_type'] == 'crafting']

# Get all nodes involved in crafting
crafting_node_ids = set(crafting_links['source']).union(set(crafting_links['target']))
crafting_points = points[points['id'].isin(crafting_node_ids)]

print(f"Crafting-only graph: {len(crafting_points)} nodes, {len(crafting_links)} edges")

# Create filtered visualization
graph_crafting = Cosmograph(
    points=crafting_points,
    links=crafting_links,
    point_id_by='id',
    point_label_by='label',
    point_size_by='size',
    point_color_by='color',
    link_source_by='source',
    link_target_by='target',
    link_color_by='color',
    link_arrows=True,
    background_color='#1a1a1a',
    show_hovered_point_label=True,
    show_top_labels=True,
)

# Uncomment to display:
graph_crafting

Crafting-only graph: 2120 nodes, 3766 edges


### Filter 2: Show Diamond-Related Items

In [10]:
# Find all nodes related to diamonds
diamond_items = points[points['label'].str.contains('Diamond', case=False, na=False)]['id'].tolist()

# Find all links connected to diamond items
diamond_links = links[
    links['source'].isin(diamond_items) | 
    links['target'].isin(diamond_items)
]

# Get all nodes involved
diamond_node_ids = set(diamond_links['source']).union(set(diamond_links['target']))
diamond_points = points[points['id'].isin(diamond_node_ids)]

print(f"Diamond-related graph: {len(diamond_points)} nodes, {len(diamond_links)} edges")

graph_diamond = Cosmograph(
    points=diamond_points,
    links=diamond_links,
    point_id_by='id',
    point_label_by='label',
    point_size_by='size',
    point_color_by='color',
    link_source_by='source',
    link_target_by='target',
    link_color_by='color',
    link_arrows=True,
    background_color='#1a1a1a',
    show_hovered_point_label=True,
    show_top_labels=True,
)

# Uncomment to display:
graph_diamond

Diamond-related graph: 88 nodes, 88 edges


### Filter 3: Custom Multi-Type Filter

Combine multiple transformation types:

In [ ]:
# Show only crafting, smelting, and smithing
selected_types = ['crafting', 'smelting', 'smithing']
filtered_links = links[links['transformation_type'].isin(selected_types)]

# Get all involved nodes
filtered_node_ids = set(filtered_links['source']).union(set(filtered_links['target']))
filtered_points = points[points['id'].isin(filtered_node_ids)]

print(f"Multi-type graph ({', '.join(selected_types)}):")
print(f"  {len(filtered_points)} nodes, {len(filtered_links)} edges")

# Uncomment to create and display:
# graph_multi = Cosmograph(
#     points=filtered_points,
#     links=filtered_links,
#     point_id_by='id',
#     point_label_by='label',
#     point_size_by='size',
#     point_color_by='color',
#     link_source_by='source',
#     link_target_by='target',
#     link_color_by='color',
#     link_arrows=True,
#     background_color='#1a1a1a',
#     show_hovered_point_label=True,
# )
# graph_multi

## Export & Sharing <a id="export"></a>

### Saving the Notebook

To save your work:
1. Use **File → Save** or press `Cmd+S` (Mac) / `Ctrl+S` (Windows/Linux)
2. The Cosmograph visualizations remain interactive in the saved notebook

### Exporting to HTML

To share your visualization:
1. **File → Save and Export Notebook As → HTML**
2. The exported HTML file includes fully interactive Cosmograph widgets
3. Share the HTML file with colleagues or publish it online

### Taking Screenshots

While Cosmograph doesn't have built-in screenshot functionality, you can:
- Use your browser's screenshot tools
- Use OS screenshot utilities (Cmd+Shift+4 on Mac, Win+Shift+S on Windows)
- Right-click and "Save image as" (may vary by browser)

## Summary & Next Steps

### What We Accomplished

✅ Loaded and parsed Minecraft transformation data  
✅ Created nodes for all items and multi-input transformations  
✅ Built a color-coded, interactive graph visualization  
✅ Explored filtering and customization options  

### Customization Ideas

- **Adjust colors**: Edit `config/graph_colors.txt` and re-run
- **Filter by item rarity**: Show only rare/valuable items
- **Analyze transformation chains**: Find the longest paths from raw materials to final products
- **Compare transformation types**: Create side-by-side views
- **Add annotations**: Use markdown cells to document interesting findings

### Learn More

- **Cosmograph Documentation**: [https://cosmograph.app/](https://cosmograph.app/)
- **Project Repository**: Check the README for more visualization options
- **Data Source**: `output/transformations.csv` contains all transformation data

---

*Happy exploring! 🎮📊*